## Import libraries

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from os import path

#%matplotlib notebook

from brainpipe.system import study
from brainpipe.feature import TF
from brainpipe.visual import *
from brainpipe.statistics import *

In [7]:
sf = 512
n_perm = 1000
f = (0.1, 150, 4, 2)  # Frequency vector: (from, to, width, step) 0.5 - 0.1
baseline = [250, 750] # Time in seconds (rest 500ms au milieu des 1s extraite)
lines = [0]
norm = 3
width, step = 50, 10 #Time vector
window_to_plot = [-500, 3000] # In seconds

## TF for all trials 

In [4]:
st = study('Olfacto')
subjects = [ 'LEFC','PIRJ', 'CHAF', 'SEMC', 'VACJ', 'MICP'] 

n_elec = {
    'CHAF' : 107,
    'VACJ' : 139, 
    'SEMC' : 107,
    'PIRJ' : 106,
    'LEFC' : 193,
    'MICP' : 105,
}

for su in subjects:
    for elec in range(n_elec[su]):
        file = su+'_E1E2_concat_all_bipo_new.npz'
        mat = np.load(path.join(st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250_5s_concatOK', file))
        x, sf, channel, label = mat['x'][elec, ...], int(mat['sf']), mat['channel'], mat['label']
        label = list(mat['label'])
        print (su, elec,)
        ntrials = x.shape[1]
        print ('size of the data : ', x.shape, 'Nb trials : ', ntrials)
        
        #Get the TF
        time = 1000 * np.arange(-1024, x.shape[0]-1024) / sf #to set the 0 in your TF
        print ('Time points: ', min(time), max(time))
        tfObj = TF(sf, x.shape[0], f=f, baseline=baseline,norm=norm, time=time, width=width, step=step,)
        xtf, pvaltf = tfObj.get((x), statmeth=None, n_perm=n_perm, maxstat=False, tail=2)
        xtf, pvaltf = 100*np.squeeze(xtf), np.squeeze(pvaltf)
        print ('TF size :', xtf.shape,) #'pvaltf : ', pvaltf.shape, pvaltf) #Freq bands * time windows

        # Plot everything that is inside the window parameter :
        timebin = np.array(tfObj.xvec)
        sl = slice(np.argmin(np.abs(timebin-window_to_plot[0])), np.argmin(np.abs(timebin-window_to_plot[1])))

        # Finally plot you power :
        title = 'TF_'+su+'_All_trials_Elec_'+str(elec)+'_'+channel[elec]+'_'+label[elec]+'_ntrials:'+str(x.shape[1])
#         th = 0.001 # toute les valeurs au dessus de ce seuil stat seront moins opaques
#         pvaltf_cut = pvaltf[:, sl]
#         mask = pvaltf_cut.copy()
#         print (mask)
#         signi_idx = mask <= th
#         mask[signi_idx] = 1
#         mask[np.invert(signi_idx)] = 0.7 #controle l'opacité du masque

#         # with stats
#         tfObj.plot2D(plt.figure(figsize=(12,10)), xtf[:, sl], cmap='viridis',  
# xvec=tfObj.xvec[sl], yvec=tfObj.yvec, xlabel='Time (ms)',
#                     vmin=-100, vmax=100, ylabel='Frequency (hz)', title=title,
#                    cblabel='Relative power modulations (%)', mask=mask, 
#pltype='imshow', resample=(0.1, 0.1),
#                     contour={'data':pvaltf_cut, 'level':[0.001,], 'colors':['w'],
#'linewidths':[1.5]},)         
#         addLines(plt.gca(), vLines=lines, vColor=['firebrick'], vWidth=[2])

        # without stats
        tfObj.plot2D(plt.figure(), xtf[:, sl], cmap='viridis',  xvec=tfObj.xvec[sl], yvec=tfObj.yvec, xlabel='Time (ms)',
                    vmin=-100, vmax=100, ylabel='Frequency (hz)', title=title,
                     cblabel='Relative power modulations (%)', pltype='imshow', resample=(0.1, 0.1),)
        addLines(plt.gca(), vLines=lines, vColor=['firebrick'] * 2, vWidth=[2] * 2)

        #Save all your plots
        rep = path.join(st.path, 'TimeFrequency/TF_Encoding_all_filter1_th40_400art_30_250/0_13Hz_No_Stats',su)
        fname = (rep + '_TF_All_trials_Odor_Elec_'+str(elec)+'_'+channel[elec]+'_'+label[elec]+'_ntrials:'+str(x.shape[1])+'.png')
        print (fname)
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.close()
    del x, sf, channel

-> Olfacto loaded


FileNotFoundError: [Errno 2] No such file or directory: '/media/karim/Datas4To/Analyses_Intra_EM_Odor/Olfacto/database/TS_E_all_cond_by_block_trigs_th40_art400_30_250_5s_concatOK/LEFC_E1E2_concat_all_bipo_new.npz'

## TF by cond - 2 blocks exploration

In [8]:
st = study('Olfacto')
subjects = [ 'LEFC','PIRJ', 'CHAF', 'SEMC', 'VACJ', 'MICP'] 
conds = ['early', 'late']

n_elec = {
    'CHAF' : 107,
    'VACJ' : 139, 
    'SEMC' : 107,
    'PIRJ' : 106,
    'LEFC' : 193,
    'MICP' : 105,
}

for su in subjects:
    for elec in range(n_elec[su]):
        for cond in conds:
            file = su+'_E1E2_concat_'+cond+'_bipo.npz'
            mat = np.load(path.join(st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250_5s_learning2blocks', file))
            x, sf, channel, label = mat['x'][elec, ...], int(mat['sf']), mat['channel'], mat['label']
            label = list(mat['label'])
            print (su, elec, cond)
            ntrials = x.shape[1]
            print ('size of the data : ', x.shape, 'Nb trials : ', ntrials)

            #Get the TF
            time = 1000 * np.arange(-1024, x.shape[0]-1024) / sf #to set the 0 in your TF
            print ('Time points: ', min(time), max(time))
            tfObj = TF(sf, x.shape[0], f=f, baseline=baseline,norm=norm, time=time, width=width, step=step,)
            xtf, pvaltf = tfObj.get((x), statmeth=None, n_perm=n_perm, maxstat=False, tail=2)
            xtf, pvaltf = 100*np.squeeze(xtf), np.squeeze(pvaltf)
            print ('TF size :', xtf.shape,) #'pvaltf : ', pvaltf.shape, pvaltf) #Freq bands * time windows

            # Plot everything that is inside the window parameter :
            timebin = np.array(tfObj.xvec)
            sl = slice(np.argmin(np.abs(timebin-window_to_plot[0])), np.argmin(np.abs(timebin-window_to_plot[1])))

            # Finally plot you power :
            title = 'TF_'+su+'_Elec_'+str(elec)+'_'+channel[elec]+'_'+label[elec]+'_'+cond+'_ntrials:'+str(x.shape[1])
    #         th = 0.001 # toute les valeurs au dessus de ce seuil stat seront moins opaques
    #         pvaltf_cut = pvaltf[:, sl]
    #         mask = pvaltf_cut.copy()
    #         print (mask)
    #         signi_idx = mask <= th
    #         mask[signi_idx] = 1
    #         mask[np.invert(signi_idx)] = 0.7 #controle l'opacité du masque

    #         # with stats
    #         tfObj.plot2D(plt.figure(figsize=(12,10)), xtf[:, sl], cmap='viridis',  xvec=tfObj.xvec[sl], yvec=tfObj.yvec, xlabel='Time (ms)',
    #                     vmin=-100, vmax=100, ylabel='Frequency (hz)', title=title,
    #                    cblabel='Relative power modulations (%)', mask=mask, pltype='imshow', resample=(0.1, 0.1),
    #                     contour={'data':pvaltf_cut, 'level':[0.001,], 'colors':['w'], 'linewidths':[1.5]},)         
    #         addLines(plt.gca(), vLines=lines, vColor=['firebrick'], vWidth=[2])

            # without stats
            tfObj.plot2D(plt.figure(), xtf[:, sl], cmap='viridis',  xvec=tfObj.xvec[sl], yvec=tfObj.yvec, xlabel='Time (ms)',
                        vmin=-100, vmax=100, ylabel='Frequency (hz)', title=title,
                         cblabel='Relative power modulations (%)', pltype='imshow', resample=(0.1, 0.1),)
            addLines(plt.gca(), vLines=lines, vColor=['firebrick'] * 2, vWidth=[2] * 2)

            #Save all your plots
            rep = path.join(st.path, 'TimeFrequency/TF_Encoding_all_filter1_th40_400art_30_250_Explo2blocks/0_150Hz_No_Stats',su)
            fname = (rep + '_TF_elec_'+str(elec)+'_'+channel[elec]+'_'+label[elec]+'_'+cond+'_explo_ntrials:'+str(x.shape[1])+'.png')
            print (fname)
            plt.savefig(fname, dpi=300, bbox_inches='tight')
            plt.close()
        del x, sf, channel

-> Olfacto loaded
LEFC 0 early
size of the data :  (2560, 29) Nb trials :  29
Time points:  -2000.0 2998.046875
TF size : (73, 252)
/media/karim/Datas4To/Analyses_Intra_EM_Odor/Olfacto/TimeFrequency/TF_Encoding_all_filter1_th40_400art_30_250_Explo2blocks/0_150Hz_No_Stats/LEFC_TF_elec_0_a2-a1_Amg-pPirT_early_explo_ntrials:29.png
LEFC 0 late
size of the data :  (2560, 28) Nb trials :  28
Time points:  -2000.0 2998.046875
TF size : (73, 252)
/media/karim/Datas4To/Analyses_Intra_EM_Odor/Olfacto/TimeFrequency/TF_Encoding_all_filter1_th40_400art_30_250_Explo2blocks/0_150Hz_No_Stats/LEFC_TF_elec_0_a2-a1_Amg-pPirT_late_explo_ntrials:28.png
LEFC 1 early
size of the data :  (2560, 29) Nb trials :  29
Time points:  -2000.0 2998.046875
TF size : (73, 252)
/media/karim/Datas4To/Analyses_Intra_EM_Odor/Olfacto/TimeFrequency/TF_Encoding_all_filter1_th40_400art_30_250_Explo2blocks/0_150Hz_No_Stats/LEFC_TF_elec_1_a3-a2_WM-Ins&Amg-pPirT_early_explo_ntrials:29.png
LEFC 1 late
size of the data :  (2560, 28

## TF trhough learning (early, middle, late)

In [ ]:
st = study('Olfacto')
subjects = ['LEFC', 'PIRJ', 'CHAF', 'SEMC', 'VACJ', 'MICP']

n_elec = {
        'CHAF':103,
        'VACJ':133,
        'SEMC':106,
        'LEFC':187,
        'MICP':101,
        'PIRJ':84,
        }

for su in subjects:
    for elec in range(0, n_elec[su], 1):
        file = su+'_E1E2_concat_all_bipo_new.npz'
        mat = np.load(path.join(st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250_5s_concatOK', file))
        x, sf, channel, label = mat['x'][elec, ...], int(mat['sf']), mat['channel'], mat['label']
        label = list(mat['label'])
        print (su, elec,)
        ntrials = x.shape[1]
        print ('size of the data : ', x.shape, 'Nb trials : ', ntrials)
        learning_files = np.array_split(x, 3, axis=1)
        print ('learning files : ', learning_files[0].shape, learning_files[1].shape,learning_files[2].shape,)
        
        for i in range (0,3,1):
            # Get TF :
            x = learning_files[i]
            print ('selected learning file :', x.shape)
            time = 1000 * np.arange(-768, x.shape[0]-768) / sf #to set the 0 in your TF
            print ('Time points: ', min(time), max(time))
            tfObj = TF(sf, x.shape[0], f=f, baseline=baseline,norm=norm, time=time, width=width, step=step,)
            xtf, pvaltf = tfObj.get((x), statmeth='permutation', n_perm=n_perm, maxstat=False, tail=2)
            xtf, pvaltf = 100*np.squeeze(xtf), np.squeeze(pvaltf)
            print ('TF size :', xtf.shape, 'pvaltf : ', pvaltf.shape, pvaltf) #Freq bands * time windows

            # Plot everything that is inside the window parameter :
            timebin = np.array(tfObj.xvec)
            sl = slice(np.argmin(np.abs(timebin-window_to_plot[0])), np.argmin(np.abs(timebin-window_to_plot[1])))

            # Finally plot you power :
            title = 'TF_'+su+'_Odor_Learning_'+str(i)+'_Elec_'+str(elec)+'_'+channel[elec]+'_'+label[elec]+'_ntrials:'+str(x.shape[1])
            th = 0.01 # toute les valeurs au dessus de ce seuil stat seront moins opaques
            pvaltf_cut = pvaltf[:, sl]
            mask = pvaltf_cut.copy()
            print (mask)
            signi_idx = mask <= th
            mask[signi_idx] = 1
            mask[np.invert(signi_idx)] = 0.6 #controle l'opacité du masque

            # with stats
            tfObj.plot2D(plt.figure(), xtf[:, sl], cmap='viridis',  xvec=tfObj.xvec[sl], yvec=tfObj.yvec, xlabel='Time (ms)',
                        vmin=-100, vmax=100, ylabel='Frequency (hz)', title=title,
                       cblabel='Relative power modulations (%)', mask=mask, pltype='imshow', resample=(0.1, 0.1),
                        contour={'data':pvaltf_cut, 'level':[0.001, 0.01], 'colors':['w', 'orange'], 'linewidths':[1.5,1]},)         
            addLines(plt.gca(), vLines=lines, vColor=['firebrick'] * 2, vWidth=[2] * 2)

            #without stats
            #tfObj.plot2D(plt.figure(), xtf[:, sl], cmap='viridis',  xvec=tfObj.xvec[sl], yvec=tfObj.yvec, xlabel='Time (ms)',
              #           vmin=-100, vmax=100, ylabel='Frequency (hz)', title=title,
              #            cblabel='Relative power modulations (%)', pltype='imshow', resample=(0.1, 0.1),)
            #addLines(plt.gca(), vLines=lines, vColor=['firebrick'] * 2, vWidth=[2] * 2)

            #Save all your plots
            rep = path.join(st.path, 'TimeFrequency/TF_Encoding_all_filter1_th40_400art_30_250/Stats_uncorrect_learning',su)
            fname = (rep + '_TF_Odor_Elec_'+str(elec)+'_'+channel[elec]+'_'+label[elec]+'_ntrials:'+str(x.shape[1])+'_Learning_'+str(i)+'.png')
            print (fname)
            plt.savefig(fname, dpi=300, bbox_inches='tight')
            plt.close()
    del x, sf, channel



